In [7]:
from backend.src.extractors.data_ingestion import DataIngestion
from backend.src.extractors.image_summarizer import ImageSummarizer
from backend.src.extractors.text_summarizer import TextSummarizer
from backend.src.constants import *
from backend.src.config.configuration import ConfigurationManager
import os

from langchain_openai import ChatOpenAI

In [4]:
import os
os.getcwd()

'/Users/goldyrana/mess/deep_learning/projects/rag/backend/notebooks'

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
os.chdir('../../') 
os.getcwd()

'/Users/goldyrana/mess/deep_learning/projects/rag'

In [11]:
test_pdf_path = "data/raw_pdfs/2023_removed.pdf"
model = ChatOpenAI(temperature=0, model="gpt-4o")

config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
data_ingestion_config = config_manager.get_data_ingestion_params()
image_summarizer_config = config_manager.get_image_summarizer_params()
text_summarizer_config = config_manager.get_text_summarizer_params()

data_ingestion = DataIngestion(data_ingestion_config)
image_summarizer = ImageSummarizer(image_summarizer_config, model)

In [ ]:
test_pdf_path = "backend/data/raw_pdfs/2023_removed.pdf"
model = ChatOpenAI(temperature=0, model="gpt-4o")

config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
data_ingestion_config = config_manager.get_data_ingestion_params()
image_summarizer_config = config_manager.get_image_summarizer_params()
text_summarizer_config = config_manager.get_text_summarizer_params()

data_ingestion = DataIngestion(data_ingestion_config)
image_summarizer = ImageSummarizer(image_summarizer_config, model)

raw_pdf_elements, report_dir = data_ingestion.process_pdf(test_pdf_path,
                                                          save=True,
                                    strategy="hi_res", # 
                                    split_pdf_page=True,  # to process each page seprately
                                    split_pdf_allow_failed=True, # continue processing even if some pages fail
                                    extract_images_in_pdf=True,
                                    infer_table_structure=True,
                                    chunking_strategy="by_title",
                                    extract_image_block_types = ["Image" , "Table"],
                                    max_characters=4000,
                                    new_after_n_chars=3800,
                                    combine_text_under_n_chars=2000)


image_path = "backend/data/reports/2023_removed/images"    
images_path = image_summarizer.get_image_path(image_path)
print(image_path)
encoded_images = []
image_summaries = []
meta_data = []

for image in images_path:
    encoded_image, image_source_path = image_summarizer.encode_image(image)
    image_summaries.append(image_summarizer.image_summarize(encoded_image))
    encoded_images.append(encoded_image)
    meta_data.append({"source": image_source_path,
                      "type": "image",
                      "title": image.split("/")[-1]})
    
image_summaries = image_summarizer.add_metadata(encoded_images, image_summaries, metadata=meta_data, automatic_metadata=False)

# # text summaries
text_summarizer = TextSummarizer(text_summarizer_config, model)
text_data = text_summarizer.get_text_data(raw_pdf_elements)

In [14]:
report_dir

'data/reports/2023_removed'

In [12]:
len(image_summaries), len(meta_data), len(encoded_images)

(10, 10, 10)

# vector store testing

In [7]:
from backend.src.storage.chroma_storage import VectorDatabase
from langchain_openai import OpenAIEmbeddings


vector_database_config = config_manager.get_vectordatabase_config()
vector_database = VectorDatabase(vector_database_config)
chroma = vector_database.init_chromadb(embeddings=OpenAIEmbeddings())
retriever = vector_database.get_multivector_retriever(chroma)
print(retriever)

2025-02-11 18:55:03,615 - chromadb.api.segment - DEBUG - Collection test5 already exists, returning existing collection.


vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x324e669a0> docstore=<langchain_core.stores.InMemoryStore object at 0x321f72fa0> search_kwargs={}


# Store text data


In [15]:
raw_text, text_summaries, summary_metadata = text_summarizer.generate_summary(text_data)

2025-02-11 18:47:21,084 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'Act as an assistant tasked with summarizing the tables and text. Give a concise summary of the table or text. Table or text chunk: Cautionary Statement\n\nThis presentation provides management\'s current operational plan for ConocoPhillips over roughly the next decade, for the assets currently in our portfolio, and is subject to multiple assumptions, including, unless otherwise specifically noted: * anoil price of $60/BBL West Texas Intermediate in 2022 dollars, escalating at 2.25% annually;\n\n* anoil price of $65/BBL Brent in 2022 dollars, escalating at 2.25% annually;\n\n* agas price of $3.75/MMBTU Henry Hub in 2022 dollars, escalating at 2.25%\n\nannually;\n\nan international gas price of $8/MMBTU Title Transfer Facility & Japan Korea Marker in 2022 dollars, escalating at 2.25% annually;\n\n* cost and capital 

In [16]:
text_summaries = text_summarizer.add_metadata(raw_text, text_summaries, summary_metadata)

In [17]:
vector_database.store_to_vb(text_summaries, retriever)

2025-02-11 18:47:54,336 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x3298a4ca0>, 'json_data': {'input': [[791, 14751, 1516, 661, 22504, 50729, 1221, 13441, 92777, 3153, 6, 25605, 3197, 369, 279, 1828, 13515, 11, 3196, 389, 1510, 12032, 323, 3892, 32946, 13, 5422, 32946, 2997, 5707, 323, 6962, 3430, 41579, 11, 449, 5707, 7729, 6041, 520, 400, 1399, 16675, 9574, 369, 4410, 8421, 61748, 323, 400, 2397, 16675, 9574, 369, 47431, 11, 323, 264, 6962, 3430, 315, 400, 18, 13, 2075, 38460, 18066, 52, 369, 18063, 27636, 11, 682, 77157, 520, 220, 17, 13, 914, 4, 30171, 13, 578, 3197, 1101, 32238, 6625, 6962, 7729, 323, 5764, 12782, 13426, 304, 8515, 6530, 51165, 1405, 8581, 13, 578, 5224, 66159, 430, 1521, 527, 4741, 31348, 41579, 3917, 311, 2349, 11, 11054, 555, 3230, 57726, 11, 323, 527, 28160, 555, 5370, 9547, 1778, 439, 2853, 71389, 323, 12782, 10396, 13], [791, 1495, 507

# Store Image data

In [18]:
vector_database.store_to_vb(image_summaries, retriever)

2025-02-11 18:48:01,978 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x35ba74ee0>, 'json_data': {'input': [[791, 2217, 17610, 315, 1403, 1584, 40099, 27393, 18200, 9659, 8336, 323, 7432, 32907, 20748, 505, 220, 1049, 16, 311, 220, 2366, 16, 382, 334, 5530, 12441, 25, 74456, 24367, 12037, 1035, 12, 3146, 56, 6830, 7339, 68063, 64341, 4430, 315, 18200, 9659, 320, 15, 4, 311, 220, 1399, 4, 4390, 12, 3146, 55, 6830, 7339, 68063, 23116, 505, 220, 1049, 16, 311, 220, 2366, 16, 627, 12, 3146, 96773, 320, 14755, 7228, 1680, 334, 720, 220, 482, 48828, 520, 220, 3971, 4, 304, 220, 1049, 16, 627, 220, 482, 65201, 2315, 311, 220, 508, 4, 555, 220, 2366, 16, 627, 12, 3146, 59828, 320, 10544, 7228, 1680, 1035, 220, 482, 48828, 520, 220, 1114, 4, 304, 220, 1049, 16, 627, 220, 482, 68148, 311, 220, 1272, 4, 555, 220, 2366, 16, 382, 334, 6107, 12441, 25, 7432, 32907, 469, 5287, 1035

2025-02-11 18:48:01,981 - openai._base_client - DEBUG - Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2025-02-11 18:48:01,983 - httpcore.connection - DEBUG - close.started
2025-02-11 18:48:01,984 - httpcore.connection - DEBUG - close.complete
2025-02-11 18:48:01,984 - httpcore.connection - DEBUG - connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
2025-02-11 18:48:01,997 - httpcore.connection - DEBUG - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x35ba69760>
2025-02-11 18:48:01,998 - httpcore.connection - DEBUG - start_tls.started ssl_context=<ssl.SSLContext object at 0x32d932ec0> server_hostname='api.openai.com' timeout=None
2025-02-11 18:48:02,015 - httpcore.connection - DEBUG - start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x35b982910>
2025-02-11 18:48:02,015 - httpcore.http11 - DEBUG - send_request_headers.started request=<Request [b'POST']>
2025

In [9]:
from backend.src.storage.chroma_storage import VectorDatabase
from backend.src.config.configuration import ConfigurationManager
from backend.src.llm_models import get_openai_embeddings

In [10]:
config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
vb_params = config_manager.get_vectordatabase_config()
print(vb_params)
vb = VectorDatabase(vb_params)
chroma = vb.init_chromadb(get_openai_embeddings())

retriever = vb.get_multivector_retriever(chroma, id_key="doc_id")


2025-02-16 22:30:52,283 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


VectorDatabaseConfig(vectorstore_name='test7', persist_directory='vector_database/vc')


2025-02-16 22:30:52,471 - chromadb.config - DEBUG - Starting component System
2025-02-16 22:30:52,471 - chromadb.config - DEBUG - Starting component Posthog
2025-02-16 22:30:52,472 - chromadb.config - DEBUG - Starting component OpenTelemetryClient
2025-02-16 22:30:52,472 - chromadb.config - DEBUG - Starting component SqliteDB
2025-02-16 22:30:52,479 - chromadb.config - DEBUG - Starting component SimpleQuotaEnforcer
2025-02-16 22:30:52,480 - chromadb.config - DEBUG - Starting component SimpleRateLimitEnforcer
2025-02-16 22:30:52,480 - chromadb.config - DEBUG - Starting component LocalSegmentManager
2025-02-16 22:30:52,480 - chromadb.config - DEBUG - Starting component LocalExecutor
2025-02-16 22:30:52,481 - chromadb.config - DEBUG - Starting component SegmentAPI
2025-02-16 22:30:52,488 - chromadb.api.segment - DEBUG - Collection test7 already exists, returning existing collection.


In [11]:
from backend.src.queries.handler import QueryHandler
qh = QueryHandler(retriever)
retriever = vb.get_vectorstore_as_retreiever(retriever.vectorstore)

In [13]:
x = retriever.invoke("hello how are ya ")

2025-02-16 22:31:15,900 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x1263488b0>, 'json_data': {'input': [[15339, 1268, 527, 13835, 220]], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
2025-02-16 22:31:15,904 - openai._base_client - DEBUG - Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2025-02-16 22:31:15,906 - httpcore.connection - DEBUG - close.started
2025-02-16 22:31:15,909 - httpcore.connection - DEBUG - close.complete
2025-02-16 22:31:15,910 - httpcore.connection - DEBUG - connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
2025-02-16 22:31:15,950 - httpcore.connection - DEBUG - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x32e9aed90>
2025-02-16 22:31:15,951 - httpcore.connection - DEBUG - start_tls.started ssl_context=<ssl.SSLContext ob

In [15]:
for i in x:
    print(i)

page_content='This is a photograph of an older man with white hair. He is wearing a dark suit, a white shirt, and a red tie. The background is plain white, and he is smiling at the camera. There are no graphs or numbers in the image.' metadata={'doc_id': '1b789dea-0086-4107-a2d9-a10e4a9fd4ef', 'raw_string': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAFZAUUDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVl

In [1]:
import os
os.chdir('../../'), os.getcwd()

(None, '/Users/goldyrana/mess/deep_learning/projects/rag')

In [2]:
from backend.src.extractors.data_ingestion import DataIngestion
from backend.src.extractors.image_summarizer import ImageSummarizer
from backend.src.extractors.text_summarizer import TextSummarizer
from backend.src.constants import *
from backend.src.config.configuration import ConfigurationManager
from backend.src.storage.chroma_storage import VectorDatabase
from backend.src.queries.handler import QueryHandler

from backend.src.llm_models import get_openai_model, get_openai_embeddings
from backend.src.constants import (CONFIG_FILE_PATH, PARAMS_FILE_PATH, FIREBASE_CREDENTIALS_PATH)

import os

/Users/goldyrana/miniconda3/envs/bounce/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dotenv import load_dotenv
load_dotenv()

model = get_openai_model()
embedding = get_openai_embeddings()


In [4]:

# get config manger
config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
data_ingestion_config = config_manager.get_data_ingestion_params()
image_summarizer_config = config_manager.get_image_summarizer_params()
text_summarizer_config = config_manager.get_text_summarizer_params()
vector_database_config = config_manager.get_vectordatabase_config()

data_ingestion = DataIngestion(data_ingestion_config)
image_summarizer = ImageSummarizer(image_summarizer_config, model)
vector_database = VectorDatabase(vector_database_config)
chroma = vector_database.init_chromadb(embeddings=get_openai_embeddings())
retriever = vector_database.get_multivector_retriever(chroma)


2025-02-16 23:09:45,714 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-02-16 23:09:45,819 - chromadb.config - DEBUG - Starting component System
2025-02-16 23:09:45,819 - chromadb.config - DEBUG - Starting component Posthog
2025-02-16 23:09:45,820 - chromadb.config - DEBUG - Starting component OpenTelemetryClient
2025-02-16 23:09:45,820 - chromadb.config - DEBUG - Starting component SqliteDB
2025-02-16 23:09:45,854 - chromadb.config - DEBUG - Starting component SimpleQuotaEnforcer
2025-02-16 23:09:45,854 - chromadb.config - DEBUG - Starting component SimpleRateLimitEnforcer
2025-02-16 23:09:45,855 - chromadb.config - DEBUG - Starting component LocalSegmentManager
2025-02-16 23:09:45,855 - chromadb.config - DEBUG - Starting component LocalExecutor
2025-02-16 23:09:45,856 - chromadb.config - DEBUG - Starting component SegmentAPI


In [5]:
# pdf file paths
raw_pdf_path = "backend/data/raw_pdfs"
pdfs = os.listdir(raw_pdf_path)
pdf_file_paths = [os.path.join(raw_pdf_path, i) for i in pdfs]


In [6]:
pdf_file_paths

['backend/data/raw_pdfs/2023_removed.pdf',
 'backend/data/raw_pdfs/2024_removed.pdf']

In [ ]:
for pdf_path in pdf_file_paths:
        
        print("processing pdf: ", pdf_path)
        raw_pdf_elements, report_dir = data_ingestion.process_pdf(pdf_path,
                                                                save=True,
                                                                strategy="hi_res", # 
                                                                split_pdf_page=True,  # to process each page seprately
                                                                split_pdf_allow_failed=True, # continue processing even if some pages fail
                                                                extract_images_in_pdf=True,
                                                                infer_table_structure=True,
                                                                chunking_strategy="by_title",
                                                                extract_image_block_types = ["Image" , "Table"],
                                                                max_characters=4000,
                                                                new_after_n_chars=3800,
                                                                combine_text_under_n_chars=2000)
        image_path = os.path.join(report_dir, "images")
        images_path = image_summarizer.get_image_path(image_path)
        
        print("working directory of images", image_path)
        print("length of images: ",  len(images_path))
        encoded_images = []
        image_summaries = []
        meta_data = []
        
        for image in images_path:
            encoded_image, image_source_path = image_summarizer.encode_image(image)
            image_summaries.append(image_summarizer.image_summarize(encoded_image))
            encoded_images.append(encoded_image)
            meta_data.append({"source": image_source_path,
                            "type": "image",
                            "title": image.split("/")[-1]})
            
        image_summaries = image_summarizer.add_metadata(encoded_images, image_summaries,
                                                        metadata=meta_data, 
                                                        automatic_metadata=False)

        # # text summaries
        text_summarizer = TextSummarizer(text_summarizer_config, model)
        text_data = text_summarizer.get_text_data(raw_pdf_elements)     
        
        # Store text data
        raw_text, text_summaries, summary_metadata = text_summarizer.generate_summary(text_data)
        text_summaries = text_summarizer.add_metadata(raw_text, text_summaries, summary_metadata)
        vector_database.store_to_vb(text_summaries, retriever)
        
        # Store image data
        vector_database.store_to_vb(image_summaries, retriever)
        
        break

In [ ]:



def main(pdf_file_paths=pdf_file_paths):
    for pdf_path in pdf_file_paths:
        
        print("processing pdf: ", pdf_path)
        raw_pdf_elements, report_dir = data_ingestion.process_pdf(pdf_path,
                                                                save=True,
                                                                strategy="hi_res", # 
                                                                split_pdf_page=True,  # to process each page seprately
                                                                split_pdf_allow_failed=True, # continue processing even if some pages fail
                                                                extract_images_in_pdf=True,
                                                                infer_table_structure=True,
                                                                chunking_strategy="by_title",
                                                                extract_image_block_types = ["Image" , "Table"],
                                                                max_characters=4000,
                                                                new_after_n_chars=3800,
                                                                combine_text_under_n_chars=2000)
        image_path = os.path.join(report_dir, "images")
        images_path = image_summarizer.get_image_path(image_path)
        
        encoded_images = []
        image_summaries = []
        meta_data = []
        
        for image in images_path:
            encoded_image, image_source_path = image_summarizer.encode_image(image)
            image_summaries.append(image_summarizer.image_summarize(encoded_image))
            encoded_images.append(encoded_image)
            meta_data.append({"source": image_source_path,
                            "type": "image",
                            "title": image.split("/")[-1]})
            
        image_summaries = image_summarizer.add_metadata(encoded_images, image_summaries,
                                                        metadata=meta_data, 
                                                        automatic_metadata=False)

        # # text summaries
        text_summarizer = TextSummarizer(text_summarizer_config, model)
        text_data = text_summarizer.get_text_data(raw_pdf_elements)     
        
        # Store text data
        raw_text, text_summaries, summary_metadata = text_summarizer.generate_summary(text_data)
        text_summaries = text_summarizer.add_metadata(raw_text, text_summaries, summary_metadata)
        vector_database.store_to_vb(text_summaries, retriever)
        
        # Store image data
        vector_database.store_to_vb(image_summaries, retriever)
        

if __name__ == "__main__":
    main()
    


# ------------------------------Testing image database alone--------------------------

In [1]:
import os
os.chdir('../../'), os.getcwd()

(None, '/Users/goldyrana/mess/deep_learning/projects/rag')

In [3]:
from backend.src.storage.chroma_storage import VectorDatabase
from backend.src.config.configuration import ConfigurationManager
from backend.src.llm_models import get_openai_embeddings



2025-02-17 16:35:20,206 - layoutparser.file_utils - DEBUG - PyTorch version 2.6.0 available.
2025-02-17 16:35:20,209 - layoutparser.file_utils - DEBUG - Effdet version 0.4.1 available.
2025-02-17 16:35:20,210 - layoutparser.file_utils - DEBUG - Pytesseract version 0.3.13 available.
2025-02-17 16:35:20,212 - layoutparser.file_utils - DEBUG - Google Cloud Vision Utils version 3.9.0 available.
2025-02-17 16:35:22,269 - pydot - DEBUG - pydot initializing
2025-02-17 16:35:22,270 - pydot - DEBUG - pydot 3.0.4
2025-02-17 16:35:22,369 - pydot.dot_parser - DEBUG - pydot dot_parser module initializing
2025-02-17 16:35:22,371 - pydot.core - DEBUG - pydot core module initializing
/Users/goldyrana/miniconda3/envs/bounce/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-17 16:35:23,357 - matplotlib - DEBUG - mat

In [4]:
from backend.src.constants import (CONFIG_FILE_PATH, PARAMS_FILE_PATH, FIREBASE_CREDENTIALS_PATH)

In [5]:
config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
vb_params = config_manager.get_vectordatabase_config()
print(vb_params)
image_vb = VectorDatabase(vb_params, persist_directory="vector_database/image")
image_chroma = image_vb.init_chromadb(get_openai_embeddings())
image_retriever = image_vb.get_multivector_retriever(image_chroma, id_key="doc_id")


VectorDatabaseConfig(vectorstore_name='test8', persist_directory='vector_database/vb1')


2025-02-17 16:35:29,764 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-02-17 16:35:29,874 - chromadb.config - DEBUG - Starting component System
2025-02-17 16:35:29,875 - chromadb.config - DEBUG - Starting component Posthog
2025-02-17 16:35:29,875 - chromadb.config - DEBUG - Starting component OpenTelemetryClient
2025-02-17 16:35:29,876 - chromadb.config - DEBUG - Starting component SqliteDB
2025-02-17 16:35:29,881 - chromadb.config - DEBUG - Starting component SimpleQuotaEnforcer
2025-02-17 16:35:29,882 - chromadb.config - DEBUG - Starting component SimpleRateLimitEnforcer
2025-02-17 16:35:29,883 - chromadb.config - DEBUG - Starting component LocalSegmentManager
2025-02-17 16:35:29,883 - chromadb.config - DEBUG - Starting component LocalExecutor
2025-02-17 16:35:29,884 - chromadb.config - DEBUG - Starting component SegmentAPI
2025-02-17 16:35:29,895 - chromadb.api.segmen

In [9]:
from backend.src.queries.handler import QueryHandler
# qh = QueryHandler(image_retriever)
image_retriever = image_vb.get_vectorstore_as_retreiever(image_retriever.vectorstore)

In [18]:
x = image_retriever.invoke("goldura aaadsf")

2025-02-17 16:38:35,168 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x33b821d30>, 'json_data': {'input': [[35637, 5808, 30109, 7819, 69]], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
2025-02-17 16:38:35,171 - openai._base_client - DEBUG - Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2025-02-17 16:38:35,175 - httpcore.connection - DEBUG - close.started
2025-02-17 16:38:35,180 - httpcore.connection - DEBUG - close.complete
2025-02-17 16:38:35,183 - httpcore.connection - DEBUG - connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
2025-02-17 16:38:35,233 - httpcore.connection - DEBUG - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x33b8807f0>
2025-02-17 16:38:35,234 - httpcore.connection - DEBUG - start_tls.started ssl_context=<ssl.SSLContext ob

In [19]:
for i in x:
    
    print(i.metadata.keys())
    print(i.metadata["type"])

dict_keys(['doc_id', 'raw_string', 'source', 'title', 'type'])
image
dict_keys(['doc_id', 'raw_string', 'source', 'title', 'type'])
image
dict_keys(['doc_id', 'raw_string', 'source', 'title', 'type'])
image
dict_keys(['doc_id', 'raw_string', 'source', 'title', 'type'])
image


## Testing text database alone


In [20]:
text_vb = VectorDatabase(vb_params, persist_directory="vector_database/text")
text_chroma = text_vb.init_chromadb(get_openai_embeddings())
text_retriever = text_vb.get_multivector_retriever(text_chroma, id_key="doc_id")

2025-02-17 16:38:38,622 - chromadb.api.segment - DEBUG - Collection test8 already exists, returning existing collection.


In [21]:
from backend.src.queries.handler import QueryHandler
qh = QueryHandler(text_retriever)
text_retriever = text_vb.get_vectorstore_as_retreiever(text_retriever.vectorstore)

In [22]:
x = text_retriever.invoke("hello how are ya")

2025-02-17 16:38:40,241 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x3309f0ee0>, 'json_data': {'input': [[15339, 1268, 527, 13835]], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
2025-02-17 16:38:40,242 - openai._base_client - DEBUG - Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2025-02-17 16:38:40,243 - httpcore.connection - DEBUG - connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
2025-02-17 16:38:40,316 - httpcore.connection - DEBUG - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x32f888250>
2025-02-17 16:38:40,316 - httpcore.connection - DEBUG - start_tls.started ssl_context=<ssl.SSLContext object at 0x33b878bc0> server_hostname='api.openai.com' timeout=None
2025-02-17 16:38:40,403 - httpcore.connection - DEBUG - start_tls.complete retu

In [23]:
for i in x:
    
    print(i.metadata.keys())
    print(i.metadata["type"])

dict_keys(['doc_id', 'file_directory', 'filename', 'filetype', 'languages', 'last_modified', 'orig_elements', 'page_number', 'raw_string', 'type'])
text
dict_keys(['doc_id', 'file_directory', 'filename', 'filetype', 'languages', 'last_modified', 'orig_elements', 'page_number', 'raw_string', 'type'])
text
dict_keys(['doc_id', 'file_directory', 'filename', 'filetype', 'languages', 'last_modified', 'orig_elements', 'page_number', 'raw_string', 'type'])
text
dict_keys(['doc_id', 'file_directory', 'filename', 'filetype', 'languages', 'last_modified', 'orig_elements', 'page_number', 'raw_string', 'type'])
text


# Sucess text is store in image and text vector database

# Testing logs

In [28]:
def log_error(exception, sucess_message=None, failure_message=None):
    """
    A decorator that logs a success message if the decorated function executes without exceptions,
    and logs an error message along with the exception details if an exception is raised.
    Args:
        sucess_message (str, optional): The message to log if the function executes successfully.
        failure_message (str, optional): The message to log if the function raises an exception.
    Returns:
        function: The decorated function with added logging functionality.
    """
    def decorator(func):  # This is the actual decorator
        def wrapper(*args, **kwargs):
            try:
                result = func(*args, **kwargs)
                if sucess_message is not None:
                    logger.name = os.path.__file__
                    logger.info(sucess_message)
                return result
            except exception as e:
                print(f"ERROR: {failure_message}\nException in {func.__name__}: {e}")
                raise
        return wrapper
    return decorator 

from backend.logger import logger

class BackendException(Exception):
    """Base exception class for all backend exceptions"""
    pass

class DataIngestionError(BackendException):
    """Exception raised for errors during data ingestion"""
    def __init__(self, message="Error occurred during data ingestion"):
        self.message = message
        super().__init__(self.message)
        
        
@log_error(DataIngestionError, sucess_message="Function executed successfully", 
           failure_message="Function failed to execute")
def foo():
    print("hello")
    raise DataIngestionError(message="Function failed to execute")

In [29]:
foo()

hello
ERROR: Function failed to execute
Exception in foo: Function failed to execute


DataIngestionError: Function failed to execute

In [30]:
from backend.logger import logger

logger.name ="test"



In [33]:
import os
os.path.__file__

'/Users/goldyrana/miniconda3/envs/bounce/lib/python3.9/posixpath.py'

In [35]:
import os

current_file_name = os.path.basename(__file__)
print(current_file_name)

NameError: name '__file__' is not defined

In [31]:
foo()

hello
ERROR: Function failed to execute
Exception in foo: Function failed to execute


DataIngestionError: Function failed to execute

In [32]:
logger.name

'test'